In [68]:
from random import random

import numpy as np
from phe import paillier
import requests

In [40]:
PRECISION = 2**(-16)
EXPONENT = -8

public_key, private_key = paillier.generate_paillier_keypair()

def get_encrypted_data(x, public_key):
    encrypted_x = [public_key.encrypt(i, precision=PRECISION) for i in x]
    encrypted_x = [i.ciphertext() for i in encrypted_x]
    return encrypted_x

In [41]:
def query_pred(x):
    url = "http://localhost:8000/prediction"
    encrypted_x = get_encrypted_data(x, public_key)
    response = requests.post(url, json={
        "pub_key_n": public_key.n,
        "enc_feature_vector": encrypted_x
    })
    
    if response.status_code == 200:
        return private_key.decrypt(paillier.EncryptedNumber(public_key, response.json()['enc_prediction'], EXPONENT))
    else:
        return response.text

In [47]:
secret_number_list = [3.141592653, 300, -4.6e-12]
encrypted_number_list = [public_key.encrypt(x, precision=PRECISION) for x in secret_number_list]

encrypted_number_list = [i.ciphertext() for i in encrypted_number_list]

exponent = -4
encrypted_number_list = [paillier.EncryptedNumber(public_key, i, exponent) for i in encrypted_number_list]

print([private_key.decrypt(i) for i in encrypted_number_list])

# What you can see in terminal is, close to secret_number_list, but limited by precision.
# >>> [3.1415863037109375, 300.0, 0.0]

[3.1415863037109375, 300.0, 0.0]


In [48]:
assert 2**(-16) > abs(query_pred([0.48555949, 0.29289251, 0.63463107,
                                  0.41933057, 0.78672205, 0.58910837,
                                  0.00739207, 0.31390802, 0.37037496,
                                  0.3375726 ]) - 0.44812144746653826)

In [101]:
from random import random

def steal_weights(dim=10):
    url = "http://localhost:8000/prediction"
    
    # Get bias
    zeros = [0.0 for _ in range(10)]
    encrypted_zeros = get_encrypted_data(zeros, public_key)
    b = private_key.decrypt(paillier.EncryptedNumber(public_key, requests.post(url, json={
        "pub_key_n": public_key.n,
        "enc_feature_vector": encrypted_zeros
    }).json()['enc_prediction'], EXPONENT))

    # Get weights
    random_features = [[random() for _ in range(10)] for _ in range(dim)]
    responses = []
    
    for req in random_features:
        encrypted_req = get_encrypted_data(req, public_key)
        response = requests.post(url, json={
            "pub_key_n": public_key.n,
            "enc_feature_vector": encrypted_req
        })
        if response.status_code == 200:
            responses.append(private_key.decrypt(paillier.EncryptedNumber(public_key, response.json()['enc_prediction'], EXPONENT)))
        else:
            print(response.text)
            return None, None
            
    if len(responses) == dim:
        features = np.array(random_features)
        labels = np.array(responses) - b
        
        weights = np.linalg.pinv(features) @ labels
        
        return weights, b

In [105]:
req = [0.48555949, 0.29289251, 0.63463107, 0.41933057, 0.78672205, 0.58910837, 0.00739207, 0.31390802, 0.37037496, 0.3375726]

weights, b = steal_weights()
local_computation = weights @ np.array(req) + b

server_computation = query_pred(req)

assert local_computation - server_computation < 1e-5